<a href="https://colab.research.google.com/github/hajer3bdullah70/MOZN_DS/blob/main/MOZN_project_DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Run this cell to generate the data
! pip install faker
import numpy as np
import pandas as pd
import uuid
import datetime

from faker.providers.person.en import Provider
from faker import Faker

from faker import Faker
fake = Faker()
Faker.seed(0)

n_employees = 50
first_names = pd.Series(list(set(Provider.first_names))).sample(n_employees)
df_employees = pd.DataFrame(columns=["id", "name", "salary"])
df_employees["name"] = (first_names).sample(frac=1).values
mu = 28_000
minimum = 16_000
df_employees["salary"] = (np.random.pareto(a=6, size=(50,1)) * mu) + minimum
df_employees["salary"] = (df_employees["salary"]/100).astype(int) * 100
df_employees["id"] = pd.Series([str(uuid.uuid4()).split('-')[1] for x in range(n_employees)])

def create_random_project(number_of_employees, duration, year, is_ongoing=False):
    df_projects = pd.DataFrame(index=range(number_of_employees), columns=["employee_id", "project_id", "start_date", "end_date"])

    employees = df_employees.id.sample(number_of_employees)

    start_date = datetime.date(year=year, month=1, day=1)
    end_date = datetime.date(year=year+1, month=1, day=1)
    project_start_date = fake.date_between(start_date=start_date, end_date=end_date)
    project_end_date = project_start_date + datetime.timedelta(days=duration)

    if is_ongoing:
        project_end_date = None

    project_id = str(uuid.uuid4()).split('-')[4]

    df_projects["employee_id"] = employees.values
    df_projects["project_id"] = project_id
    df_projects["start_date"] = project_start_date
    df_projects["end_date"] = project_end_date


    return df_projects

n_projects = 150
projects = []
for proj in range(n_projects):
    number_of_employees = np.random.choice([1,2,3,4], replace=True, p=[0.25, 0.5, 0.15, 0.1], size=1)
    duration = (np.random.pareto(a=5, size=1) * 30*4) + 14
    year = np.random.choice([2017, 2018, 2019, 2020])
    is_ongoing = bool(np.random.binomial(n=1, p=0.4)) if year >= 2019 else False
    new_project = (create_random_project(number_of_employees=number_of_employees[0], duration=duration[0], year=year, is_ongoing=is_ongoing).values)

    projects.append(new_project)

projects_data = np.vstack(projects)
df_projects = (
    pd.DataFrame(projects_data, columns=["employee_id", "project_id", "start_date", "end_date"])
    .sort_values(["project_id", "start_date", "end_date"])
    .reset_index(drop=True)
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.7 MB/s eta 0:00:00


In [2]:
df_employees.head(5)

,id,name,salary
0,766d,Darcie,22800
1,287f,Justice,16800
2,6175,Tayla,23200
3,ef3c,Bose,24000
4,a758,Ebbie,23900


In [3]:
df_projects.head(5)

,employee_id,project_id,start_date,end_date
0,3358,025513544622,2018-07-27,2018-08-17
1,aa27,025513544622,2018-07-27,2018-08-17
2,c3a1,085b0c810855,2019-06-05,2019-08-02
3,836f,085b0c810855,2019-06-05,2019-08-02
4,6754,085b0c810855,2019-06-05,2019-08-02


**bold text**#####
## Find the 3 lowest paid employees who have done at least 10 projects.

## Write your solution here, and copy it back to the form

---



In [4]:
import pandas as pd

#Count number of projects per employee
project_counts = (
    df_projects.groupby("employee_id")
    .size()
    .reset_index(name="project_count")
)

#Filter employees with at least 10 projects
qualified = project_counts[project_counts["project_count"] >= 10]

# Merge with employee data to get salary
merged = pd.merge(qualified, df_employees, left_on="employee_id", right_on="id")

#Sort by salary and select the 3 lowest-paid
lowest_paid = merged.sort_values(by="salary").head(3)

#Final output
print("3 Lowest-paid employees with at least 10 projects:")
print(lowest_paid[["id", "name", "salary", "project_count"]])


3 Lowest-paid employees with at least 10 projects:
     id   name  salary  project_count
1  6754  Edwin   16600             10
5  d2fb  Tegan   16600             10
3  bab4    Ras   17000             10
